**Still a work in progress**. The current [leaderboard](https://www.kaggle.com/c/dog-breed-identification/leaderboard) score is 0.551.

In [1]:
import numpy as np
import pandas as pd
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

Using TensorFlow backend.


In [2]:
# load the labels of train set (generated by "get_features.py")
Y_train = np.load('train_labels.npy')

In [3]:
# load the bottleneck features (generated by "get_features.py")
X_train = np.load('bottleneck_features_train.npy')
X_test = np.load('bottleneck_features_test.npy')

In [4]:
X_train.shape, Y_train.shape, X_test.shape

((10222, 8, 8, 1536), (10222, 120), (10357, 8, 8, 1536))

In [5]:
# build a model architecture
InceptionResNetV2_model = Sequential()

InceptionResNetV2_model.add(Flatten(input_shape = X_train.shape[1:]))
InceptionResNetV2_model.add(Dropout(0.5))
InceptionResNetV2_model.add(Dense(240, activation='relu'))
InceptionResNetV2_model.add(Dropout(0.5))
InceptionResNetV2_model.add(Dense(120, activation='softmax'))

InceptionResNetV2_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 98304)             0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 98304)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 240)               23593200  
_________________________________________________________________
dropout_2 (Dropout)          (None, 240)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 120)               28920     
Total params: 23,622,120
Trainable params: 23,622,120
Non-trainable params: 0
_________________________________________________________________


In [6]:
# compile and train
InceptionResNetV2_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
InceptionResNetV2_model.fit(X_train, Y_train, validation_split=0.1, epochs=10, batch_size=512, verbose=1)

Train on 9199 samples, validate on 1023 samples
Epoch 1/10
9199/9199 [==============================] - 48s 5ms/step - loss: 3.1867 - acc: 0.3350 - val_loss: 0.7825 - val_acc: 0.8319
Epoch 2/10
9199/9199 [==============================] - 47s 5ms/step - loss: 1.1628 - acc: 0.7150 - val_loss: 0.5372 - val_acc: 0.8827
Epoch 3/10
9199/9199 [==============================] - 47s 5ms/step - loss: 0.7653 - acc: 0.8154 - val_loss: 0.4979 - val_acc: 0.8866
Epoch 4/10
9199/9199 [==============================] - 50s 5ms/step - loss: 0.6132 - acc: 0.8449 - val_loss: 0.5166 - val_acc: 0.8993
Epoch 5/10
9199/9199 [==============================] - 44s 5ms/step - loss: 0.4901 - acc: 0.8753 - val_loss: 0.5217 - val_acc: 0.9032
Epoch 6/10
9199/9199 [==============================] - 44s 5ms/step - loss: 0.4164 - acc: 0.8886 - val_loss: 0.5465 - val_acc: 0.8935
Epoch 7/10
9199/9199 [==============================] - 43s 5ms/step - loss: 0.3303 - acc: 0.9064 - val_loss: 0.5656 - val_acc: 0.8944
Epoch 8

In [7]:
# make predictions and generate a csv file
Y_pred = InceptionResNetV2_model.predict(X_test)
df = pd.read_csv('sample_submission.csv')
df.iloc[:,1:] = Y_pred
df.to_csv('prediction.csv', index=False)